# English and Spanish used car sales descriptions using REST<br/>Text generation and translation scenarios

This sample will generate a sales car descriptions for used cars. It will mock getting structured data from a database call and turing that data into a simple description that then is sent to a GPT API call to create a rich sales description in English. A second GPT call translates the sales description into Spanish. This demo is based on the following user story:

- https://customers.microsoft.com/en-us/story/1501304071775762777-carmax-retailer-azure-openai-service

Learning objectives:

- Understand what I have to provide to make this REST demo work including:
  - The prompt and completion models to make the API calls
  - The code and knowledge of using the HttpClient to make make rest calls and make them resilient
  - The disjoined functions to hadle sales description and translation
  - The Code to handle submit the prompt and process the completion

## Load the required .NET packages and supporting classes

In [1]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.Extensions.Logging.Console"

using System;
using System.Net;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using dotenv.net;

#!import Models/Models.cs
#!import Utils/HttpUtils.cs
#!import Utils/ConsoleLogger.cs

Installed Packages dotenv.net, 3.1.2 Microsoft.Extensions.Logging.Console, 7.0.0

## Read the OpenAI endpoint and API key from environment variables or an .env file

In [2]:
DotEnv.Load();
var uri = Environment.GetEnvironmentVariable("GPT_OPENAI_FULL_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment at: {uri} with key {apiKey.Substring(0, 5)}...");

Using deployment at: https://alemorfr.openai.azure.com/openai/deployments/gpt/chat/completions?api-version=2023-03-15-preview with key 32c87...


## Prepare the HttpClient

- The HttpClient is the C# network object that handles HTTP/S calls.

In [3]:
// Get an HttpClient Instance
var client = new HttpClient();
client.DefaultRequestHeaders.TryAddWithoutValidation("api-key",apiKey);

## Mock a database call to get the car details

In [4]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };

List<Car> MockDBCall() {  
  return mockCarData;
}

## Get a vehicle description from the car properties

This function turns the structured data into a sentence.

In [5]:
// Reminder: I created this function, to make more impact, but the car record has a ToString defition to return the same information
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Use GPT to get a vehicle sales description

In [6]:
async Task<string> GetSalesDescription(HttpClient client, string desc) {
    var prompt = $"Get a sales description for the following car:\n{desc}";
    var chatHistory = new List<Message>();
    chatHistory.Add(new Message("user", prompt));
    var ans = await GetGptCompletionAsync(client,uri,chatHistory);
    return ans.Item1;
}

## Use GPT to translate from English to Spanish

In [7]:
async Task<string> TranslateIntoES(HttpClient client, string translate) {
    var prompt = $"Translate the following text from English to Spanish:\n{translate}";
    var chatHistory = new List<Message>();
    chatHistory.Add(new Message("user", prompt));
    var ans = await GetGptCompletionAsync(client,uri,chatHistory);
    return ans.Item1;
}

## Process each used car for a sales description, translate into Spanish,<br/>and keep a list of the work

In [10]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();

var processedCars = new List<CarTranslation>();
foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing: {desc}");
  
  // Get a sales description from Davinci
  var result = await GetSalesDescription(client, desc);
  Console.WriteLine($"Sales Description: {result}");

  
  //Translate into Spanish
  var esResult = await TranslateIntoES(client,result);
  Console.WriteLine($"Translation: {result}");
  
  // Add processed car to list
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), esResult.ToString()));
}

// Show the processed cars
processedCars

Processing: 2015 Ford Explorer Silver V6 Platinum with 10000 miles for $16500
Sales Description: Looking for a sleek and stylish SUV that delivers top-notch performance and comfort? Look no further than the 2015 Ford Explorer Silver V6 Platinum! With only 10000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The powerful V6 engine delivers impressive acceleration and handling, while the Platinum trim level offers a host of premium features like leather upholstery, heated and cooled front seats, and a premium sound system. And at just $16500, this Explorer is an unbeatable value that won't break the bank. So why wait? Come test drive the 2015 Ford Explorer Silver V6 Platinum today and experience the ultimate in SUV luxury and performance!
Translation: Looking for a sleek and stylish SUV that delivers top-notch performance and comfort? Look no further than the 2015 Ford Explorer Silver V6 Platinum! With only 10000 miles on the odometer, this vehicl

index value 0 CarTranslation { VIN = 123, En = Looking for a sleek and stylish SUV that delivers top-notch performance and comfort? Look no further than the 2015 Ford Explorer Silver V6 Platinum! With only 10000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The powerful V... VIN 123 En Looking for a sleek and stylish SUV that delivers top-notch performance and comfort? Look no further than the 2015 Ford Explorer Silver V6 Platinum! With only 10000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The powerful V6 engine delivers impressive acceleration and handling, while the Platinum trim level offers a host of premium features like leather upholstery, heated and cooled front seats, and a premium sound system. And at just $16500, this Explorer is an unbeatable value that won't break the bank. So why wait? Come test drive the 2015 Ford Explorer Silver V6 Platinum today and experience the ultimate in SUV luxury and performance! ES ¿Buscas un SUV elegante y con estilo que ofrezca un rendimiento y comodidad de primera? ¡No busques más allá del Ford Explorer Silver V6 Platinum 2015! Con solo 10000 millas en el odómetro, este vehículo es prácticamente nuevo y está listo para salir a la carretera. El potente motor V6 ofrece una impresionante aceleración y manejo, mientras que el nivel de acabado Platinum ofrece una gran cantidad de características premium como tapicería de cuero, asientos delanteros con calefacción y refrigeración y un sistema de sonido premium. Y a solo $16500, este Explorer es una oferta imbatible que no romperá el banco. Entonces, ¿por qué esperar? ¡Ven a probar el Ford Explorer Silver V6 Platinum 2015 hoy y experimenta lo último en lujo y rendimiento de SUV! 1 CarTranslation { VIN = 456, En = Introducing the ultimate driving machine - the 2018 Ford Mustang Blue V8 Sports! This stunning car is a true masterpiece, boasting a powerful V8 engine that delivers an exhilarating driving experience. With only 10000 miles on the clock, this Mustang is practically b... VIN 456 En Introducing the ultimate driving machine - the 2018 Ford Mustang Blue V8 Sports! This stunning car is a true masterpiece, boasting a powerful V8 engine that delivers an exhilarating driving experience. With only 10000 miles on the clock, this Mustang is practically brand new and ready to hit the open road. The sleek blue exterior is sure to turn heads wherever you go, while the luxurious interior is packed with all the latest features and technology. And at just $250000, this Mustang is an absolute steal. Don't miss your chance to own this incredible car - come test drive it today! ES ¡Presentamos la máquina de conducción definitiva: el Ford Mustang Blue V8 Sports 2018! Este impresionante automóvil es una verdadera obra maestra, con un potente motor V8 que ofrece una experiencia de conducción emocionante. Con solo 10000 millas en el odómetro, este Mustang es prácticamente nuevo y está listo para recorrer la carretera abierta. El elegante exterior azul seguramente llamará la atención donde quiera que vaya, mientras que el lujoso interior está equipado con todas las últimas características y tecnología. Y por solo $250000, este Mustang es un verdadero robo. ¡No pierda la oportunidad de ser dueño de este increíble automóvil, venga a probarlo hoy mismo! 2 CarTranslation { VIN = 789, En = Introducing the 2020 Ford Escape Red V6 Special, the perfect combination of style, power, and affordability! With only 3000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The sleek red exterior is sure to turn heads, while the... VIN 789 En Introducing the 2020 Ford Escape Red V6 Special, the perfect combination of style, power, and affordability! With only 3000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The sleek red exterior is sure to turn heads, while the powerful V6 engine provides a thrilling driving